## CONFIGURE PRIMARY CLUSTER

In [2]:
%%bash

export VAULT_ADDR=$(terraform output -raw cluster1_fqdn_8200)
export VAULT_TOKEN=

vault write -f sys/replication/dr/primary/enable primary_cluster_addr=$(terraform output -raw cluster1_fqdn_8201)
sleep 5

vault write -format=json sys/replication/dr/primary/secondary-token id=dr12 | jq -r .wrap_info.token > dr_token.txt



WARNING! The following warnings were returned from Vault:

  * This cluster is being enabled as a primary for replication. Vault will be
  unavailable for a brief period and will resume service shortly.



## CONFIGURE SECONDARY CLUSTER

In [3]:
%%bash
export VAULT_TOKEN=
export VAULT_ADDR=$(terraform output -raw cluster2_fqdn_8200)

vault write sys/replication/dr/secondary/enable  \
    primary_api_addr=$(terraform output -raw cluster1_fqdn_8200) \
    token=$(cat dr_token.txt)

WARNING! The following warnings were returned from Vault:

  * Vault has successfully found secondary information; it may take a while to
  perform setup tasks. Vault will be unavailable until these tasks and initial
  sync complete.



In [1]:
%%bash
export VAULT_ADDR=$(terraform output -raw cluster1_fqdn_8200)
export VAULT_TOKEN=hvs.gw2jGbxgnvR7PjDRwDi9yf21

vault policy write dr-secondary-promotion - <<EOF
path "sys/replication/dr/secondary/promote" {
  capabilities = [ "update" ]
}

# To update the primary to connect
path "sys/replication/dr/secondary/update-primary" {
    capabilities = [ "update" ]
}

# Only if using integrated storage (raft) as the storage backend
# To read the current autopilot status
path "sys/storage/raft/autopilot/state" {
    capabilities = [ "update" , "read" ]
}

path "sys/storage/raft/*" {
    capabilities = [ "update" , "read", "create", "delete", "patch", "sudo" ]
}
EOF

vault write auth/token/roles/failover-handler \
    allowed_policies=dr-secondary-promotion \
    orphan=true \
    renewable=false \
    token_type=batch


vault token create -role=failover-handler -ttl=8h

Error uploading policy: Error making API request.

URL: PUT https://vault-primary-europe-southwest1-8y03.josemerchan-8d4c7e.gcp.sbx.hashicorpdemo.com:8200/v1/sys/policies/acl/dr-secondary-promotion
Code: 400. Errors:

* failed to parse policy: path "sys/storage/raft/*": invalid capability "write"


Success! Data written to: auth/token/roles/failover-handler


WARNING! The following warnings were returned from Vault:

  * Policy "dr-secondary-promotion" does not exist



Key                  Value
---                  -----
token                hvb.AAAAAQIYN4gh0l1A5hgSKq0C0e89SZOQS3alA-_sfqCjSMsLBn2csWOgjSRehmg2Axk25QPAfZxsLWi7R_UeWKifvKOIiKIVvVo6InHulscy51mfGhGWfrjVU7RjROpBl6s-XdFERTx5h7LEu08xLxvd7BnrwR6W7et210JbWCnjkd266vqvatOsh5aQN63JNn_txS5ifw
token_accessor       n/a
token_duration       8h
token_renewable      false
token_policies       ["default" "dr-secondary-promotion"]
identity_policies    []
policies             ["default" "dr-secondary-promotion"]
